In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, avg, when
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree

import time
import sys
import os
import glob
import hdf5_getters
import re

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.79:7077") \
        .appName("SongHotness")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",1)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext
sqlContext = SQLContext(spark_context)

def rowUnpack(df):
        results={}
        for i in df:
            results.update(i.asDict())
        return(results)

In [2]:
class Song:
    songCount = 0
    # songDictionary = {}

    def __init__(self, songID):
        self.id = songID
        Song.songCount += 1
        # Song.songDictionary[songID] = self

        self.albumName = None
        self.albumID = None
        self.artistID = None
        self.artistLatitude = None
        self.artistLocation = None
        self.artistLongitude = None
        self.artistName = None
        self.danceability = None
        self.duration = None
        self.energy = None
        self.genreList = []
        self.keySignature = None
        self.keySignatureConfidence = None
        self.lyrics = None
        self.popularity = None
        self.songhotttnesss = None
        self.tempo = None
        self.timeSignature = None
        self.timeSignatureConfidence = None
        self.title = None
        self.year = None

    def displaySongCount(self):
        print("Total Song Count %i" % Song.songCount)

    def displaySong(self):
        print("ID: %s" % self.id)  
    
    

In [3]:
    
"""
Original:

Alexis Greenstreet (October 4, 2015) University of Wisconsin-Madison

This code is designed to convert the HDF5 files of the Million Song Dataset
to a CSV by extracting various song properties.

The script writes to a "SongCSV.csv" in the directory containing this script.

Please note that in the current form, this code only extracts the following
information from the HDF5 files:
AlbumID, AlbumName, ArtistID, ArtistLatitude, ArtistLocation,
ArtistLongitude, ArtistName, Danceability, Duration, KeySignature,
KeySignatureConfidence, SongID, Tempo, TimeSignature,
TimeSignatureConfidence, Title, and Year.

This file also requires the use of "hdf5_getters.py", written by
Thierry Bertin-Mahieux (2010) at Columbia University

Credit:
This HDF5 to CSV code makes use of the following example code provided
at the Million Song Dataset website 
(Home>Tutorial/Iterate Over All Songs, 
http://labrosa.ee.columbia.edu/millionsong/pages/iterate-over-all-songs),
Which gives users the following code to get all song titles:

#import os
#import glob
#import hdf5_getters
#def get_all_titles(basedir,ext='.h5') :
#    titles = []
#   for root, dirs, files in os.walk(basedir):
#        files = glob.glob(os.path.join(root,'*'+ext))
#        for f in files:
#            h5 = hdf5_getters.open_h5_file_read(f)
#            titles.append( hdf5_getters.get_title(h5) )
#            h5.close()
#   return titles 
"""

"""This code has been modified for python3 compatibility by Karthik Nair(28 May 2019) of Uppsala University (github.com/karnair)"""
"""This code has been adapted for conversion of .h5 files to PySpark Dataframe by Karthik Nair(3 June 2019) of Uppsala University (github.com/karnair)"""



'This code has been adapted for conversion of .h5 files to PySpark Dataframe by Karthik Nair(3 June 2019) of Uppsala University (github.com/karnair)'

In [4]:
def h5toPysDf():
    
   
    song_number = []
    album_id = []
    artist_latitude = []
    artist_location = []
    artist_longitude = []
    artist_name =[]
    danceability = []
    duration = []
    energy = []
    key_signature = []
    key_signature_confidence = []
    song_id = []
    tempo = []
    song_hotttnesss = []
    time_signature = []
    time_signature_confidence = []
    title = []
    year = []
    
    
    
    csvRowString = ("SongID,AlbumID,ArtistLatitude,ArtistLocation,"+
            "ArtistLongitude,ArtistName,Danceability,Duration,Energy,KeySignature,"+
            "KeySignatureConfidence,SongHotttnesss,Tempo,TimeSignature,TimeSignatureConfidence,"+
            "Title,Year")
    
    csvAttributeList = re.split('\W+', csvRowString)
    
    for i, v in enumerate(csvAttributeList):
        csvAttributeList[i] = csvAttributeList[i].lower()
        csvRowString = ""
    
    basedir = "/home/ubuntu/MillionSongSubset/data" # "." As the default means the current directory
    ext = ".h5"
    
    for root, dirs, files in os.walk(basedir):        
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            print(f)

            songH5File = hdf5_getters.open_h5_file_read(f)
            song = Song(str(hdf5_getters.get_song_id(songH5File)))

            testDanceability = hdf5_getters.get_danceability(songH5File)
            # print type(testDanceability)
            # print ("Here is the danceability: ") + str(testDanceability)

            #song.artistID = str(hdf5_getters.get_artist_id(songH5File))
            song.albumID = str(hdf5_getters.get_release_7digitalid(songH5File))
            #song.albumName = str(hdf5_getters.get_release(songH5File))
            song.artistLatitude = str(hdf5_getters.get_artist_latitude(songH5File))
            song.artistLocation = str(hdf5_getters.get_artist_location(songH5File))
            song.artistLongitude = str(hdf5_getters.get_artist_longitude(songH5File))
            song.artistName = str(hdf5_getters.get_artist_name(songH5File))
            song.danceability = float(hdf5_getters.get_danceability(songH5File))
            song.duration = float(hdf5_getters.get_duration(songH5File))
            song.energy = float(hdf5_getters.get_energy(songH5File))
            # song.setGenreList()
            song.keySignature = float(hdf5_getters.get_key(songH5File))
            song.keySignatureConfidence = float(hdf5_getters.get_key_confidence(songH5File))
            # song.lyrics = None
            # song.popularity = None
            song.tempo = float(hdf5_getters.get_tempo(songH5File))
            song.songhotttnesss = float(hdf5_getters.get_song_hotttnesss(songH5File))
            song.timeSignature = float(hdf5_getters.get_time_signature(songH5File))
            song.timeSignatureConfidence = float(hdf5_getters.get_time_signature_confidence(songH5File))
            song.title = str(hdf5_getters.get_title(songH5File))
            song.year = str(hdf5_getters.get_year(songH5File))
            
            
            
            #csvRowString += str(song.songCount) + ","
            song_number.append(song.songCount)

            for attribute in csvAttributeList:
                # print "Here is the attribute: " + attribute + " \n"

                if attribute == 'AlbumID'.lower():
                    #csvRowString += song.albumID
                    album_id.append(song.albumID)
                elif attribute == 'ArtistLatitude'.lower():
                    latitude = song.artistLatitude
                    if latitude == 'nan':
                        latitude = ''
                    artist_latitude.append(latitude)
                elif attribute == 'ArtistLocation'.lower():
                    location = song.artistLocation
                    location = location.replace(',','')
                    artist_location.append(location) 
                elif attribute == 'ArtistLongitude'.lower():
                    longitude = song.artistLongitude
                    if longitude == 'nan':
                        longitude = ''
                    artist_longitude.append(longitude)   
                elif attribute == 'ArtistName'.lower():
                    artist_name.append(song.artistName)
                elif attribute == 'Danceability'.lower():
                    danceability.append(song.danceability)
                elif attribute == 'Duration'.lower():
                    duration.append(song.duration)
                elif attribute == 'Energy'.lower():
                    energy.append(song.energy)
                elif attribute == 'KeySignature'.lower():
                    key_signature.append(song.keySignature)
                elif attribute == 'KeySignatureConfidence'.lower():
                    # print "key sig conf: " + song.timeSignatureConfidence                                 
                    key_signature_confidence.append(song.keySignatureConfidence)
                elif attribute == 'SongID'.lower():
                    song_id.append(song.id)
                elif attribute == 'Tempo'.lower():
                    # print "Tempo: " + song.tempo
                    tempo.append(song.tempo)
                elif attribute == 'SongHotttnesss'.lower():
                    song_hotttnesss.append(song.songhotttnesss)
                elif attribute == 'TimeSignature'.lower():
                    time_signature.append(song.timeSignature)
                elif attribute == 'TimeSignatureConfidence'.lower():
                    # print "time sig conf: " + song.timeSignatureConfidence                                   
                    time_signature_confidence.append(song.timeSignatureConfidence)
                elif attribute == 'Title'.lower():
                    title.append(song.title)
                elif attribute == 'Year'.lower():
                    year.append(song.year)
                #"""else:
                 #   csvRowString += "Erm. This didn't work. Error. :( :(\n" """"

                #csvRowString += ","
            songH5File.close()
    pysp_df = sqlContext.createDataFrame(zip(song_number, album_id, artist_latitude, artist_location, artist_longitude, artist_name, danceability, duration, energy, key_signature, key_signature_confidence, song_id, tempo, song_hotttnesss, time_signature, time_signature_confidence, title, year), schema=['song_number', 'album_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'danceability', 'duration', 'energy', 'key_signature', 'key_signature_confidence', 'song_id', 'tempo', 'song_hotttnesss', 'time_signature', 'time_signature_confidence', 'title', 'year'])
    return pysp_df


In [ ]:
#song_number, album_id, artist_latitude, artist_location, artist_longitude, artist_name, danceability, duration, energy, key_signature, key_signature_confidence, song_id, tempo, song_hotttnesss, time_signature, time_signature_confidence, title, year = main()
msd_df = h5toPysDf()

/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTUV128F933D382.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTJH128F1476BC7.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTSI128E0790E7B.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTDV128F934F6B1.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTQB128F92FAB03.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTFO128EF345397.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTZW12903CE7495.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTJG12903CD5B47.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTOM128F42309C2.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTUD128F42A10A3.h5
/home/ubuntu/MillionSongSubset/data/B/B/T/TRBBTLA128F4226907.h5
/home/ubuntu/MillionSongSubset/data/B/B/S/TRBBSFL128F931798C.h5
/home/ubuntu/MillionSongSubset/data/B/B/S/TRBBSBF128F14687A4.h5
/home/ubuntu/MillionSongSubset/data/B/B/S/TRBBSKC128F1496C4E.h5
/home/ubuntu/MillionSongSubset/data/B/B/S/TRBBSGI128F9309855.h5
/home/ubuntu/MillionSongSubset/data/B/B/

/home/ubuntu/MillionSongSubset/data/B/B/V/TRBBVFT128F42ABB02.h5
/home/ubuntu/MillionSongSubset/data/B/B/V/TRBBVBM128F4276D82.h5
/home/ubuntu/MillionSongSubset/data/B/B/V/TRBBVAH128F4270300.h5
/home/ubuntu/MillionSongSubset/data/B/B/V/TRBBVXL128F427C3F2.h5
/home/ubuntu/MillionSongSubset/data/B/B/V/TRBBVPM12903CDB094.h5
/home/ubuntu/MillionSongSubset/data/B/B/V/TRBBVTH128F92F3B6F.h5
/home/ubuntu/MillionSongSubset/data/B/B/V/TRBBVYO128F146AEDC.h5
/home/ubuntu/MillionSongSubset/data/B/B/R/TRBBRJN128F425C040.h5
/home/ubuntu/MillionSongSubset/data/B/B/R/TRBBRME128F4276C73.h5
/home/ubuntu/MillionSongSubset/data/B/B/R/TRBBRYI128F42370F5.h5
/home/ubuntu/MillionSongSubset/data/B/B/R/TRBBRFQ12903CE2997.h5
/home/ubuntu/MillionSongSubset/data/B/B/R/TRBBRVT128F428EA7A.h5
/home/ubuntu/MillionSongSubset/data/B/B/R/TRBBRIQ128F9353B5C.h5
/home/ubuntu/MillionSongSubset/data/B/B/R/TRBBRQE12903C9E587.h5
/home/ubuntu/MillionSongSubset/data/B/B/R/TRBBREF128F933CCDD.h5
/home/ubuntu/MillionSongSubset/data/B/B/

/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBANU128F422B87B.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBADO12903CEE832.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBAHD128F428E0FE.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBANR128F4277C5B.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBASV128F427680E.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBAQJ128F92F1336.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBAAZ12903CAEA6D.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBADQ128F933698F.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBAIC12903D11771.h5
/home/ubuntu/MillionSongSubset/data/B/B/A/TRBBAKQ128F92F0A8E.h5
/home/ubuntu/MillionSongSubset/data/B/E/T/TRBETHV128F429722F.h5
/home/ubuntu/MillionSongSubset/data/B/E/T/TRBETVK128F4288B49.h5
/home/ubuntu/MillionSongSubset/data/B/E/T/TRBETSW12903CA6A6B.h5
/home/ubuntu/MillionSongSubset/data/B/E/T/TRBETRA128F9327DC6.h5
/home/ubuntu/MillionSongSubset/data/B/E/S/TRBESXG128F4243BFB.h5
/home/ubuntu/MillionSongSubset/data/B/E/

/home/ubuntu/MillionSongSubset/data/B/E/V/TRBEVTM128F92FE2B2.h5
/home/ubuntu/MillionSongSubset/data/B/E/V/TRBEVJC12903CCE75B.h5
/home/ubuntu/MillionSongSubset/data/B/E/V/TRBEVND128F93429DD.h5
/home/ubuntu/MillionSongSubset/data/B/E/V/TRBEVZN128F9330A0D.h5
/home/ubuntu/MillionSongSubset/data/B/E/V/TRBEVSH128F92D5CFC.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBEROX128F4270745.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERKY128F42290CA.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERHG128F933AC46.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERUM128F148AC93.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERCZ128E0787D93.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERLZ12903CC4747.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERHS128F92FF079.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERGT128F42A009F.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERJE128F425F098.h5
/home/ubuntu/MillionSongSubset/data/B/E/R/TRBERIX12903CEF907.h5
/home/ubuntu/MillionSongSubset/data/B/E/

/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBLK128F4264240.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBKU128F422A0E3.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBLJ12903CCF436.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBAQ128F93279F5.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBFY128F932C0E3.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBTE128E078E882.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBSN128F92FBA64.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBTU128F92FDC9D.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBSK128F92E5624.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBVM128F4262E9D.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBKA128F42947BF.h5
/home/ubuntu/MillionSongSubset/data/B/I/B/TRBIBOJ128F14618D4.h5
/home/ubuntu/MillionSongSubset/data/B/I/J/TRBIJLT12903CE7070.h5
/home/ubuntu/MillionSongSubset/data/B/I/J/TRBIJRN128F425F3DD.h5
/home/ubuntu/MillionSongSubset/data/B/I/J/TRBIJMU12903CF892B.h5
/home/ubuntu/MillionSongSubset/data/B/I/

...

/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJEIK128F92F19EA.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJENI128F425B7F4.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJEUV12903D15D6D.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJESP12903CBC0B0.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJEYM12903CDEF6A.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJEEW128F4280555.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJENI128F9300B02.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJEDV128F93218B9.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJEOD12903CC065D.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJEZT128E0795DF3.h5
/home/ubuntu/MillionSongSubset/data/A/J/E/TRAJEYL128F931B518.h5
/home/ubuntu/MillionSongSubset/data/A/J/N/TRAJNYU128E07856A5.h5
/home/ubuntu/MillionSongSubset/data/A/J/N/TRAJNCR12903CEBAA1.h5
/home/ubuntu/MillionSongSubset/data/A/J/N/TRAJNBE128F4255C97.h5
/home/ubuntu/MillionSongSubset/data/A/J/N/TRAJNWY128F1453771.h5
/home/ubuntu/MillionSongSubset/data/A/J/

/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZRG128F930662C.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZJQ128F423F44F.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZKF12903CDA164.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZXZ128F4285158.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZCH128F933D2B7.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZWM128F4262BBD.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZYI128F424B412.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZIS128F4288E43.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZDD128F425AA57.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZDQ128F4260AC0.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZLM128F427CEB7.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZCR12903CC8D33.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZBD128F42A2502.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZII128F9358893.h5
/home/ubuntu/MillionSongSubset/data/A/J/Z/TRAJZZC128F424A4D0.h5
/home/ubuntu/MillionSongSubset/data/A/J/

/home/ubuntu/MillionSongSubset/data/A/E/J/TRAEJAC128F427FBDE.h5
/home/ubuntu/MillionSongSubset/data/A/E/J/TRAEJWK128F149EA60.h5
/home/ubuntu/MillionSongSubset/data/A/E/E/TRAEEMP128F92F4B9C.h5
/home/ubuntu/MillionSongSubset/data/A/E/E/TRAEEGO12903CF7D27.h5
/home/ubuntu/MillionSongSubset/data/A/E/E/TRAEEBP128EF3673A7.h5
/home/ubuntu/MillionSongSubset/data/A/E/E/TRAEELO128F425BD8F.h5
/home/ubuntu/MillionSongSubset/data/A/E/E/TRAEEGA128F4272FF8.h5
/home/ubuntu/MillionSongSubset/data/A/E/E/TRAEEOQ128F9309BC0.h5
/home/ubuntu/MillionSongSubset/data/A/E/E/TRAEELW128F933D255.h5
/home/ubuntu/MillionSongSubset/data/A/E/N/TRAENEO128F426789F.h5
/home/ubuntu/MillionSongSubset/data/A/E/N/TRAENGY128F4272846.h5
/home/ubuntu/MillionSongSubset/data/A/E/N/TRAENGX12903CBDFF6.h5
/home/ubuntu/MillionSongSubset/data/A/E/N/TRAENWI12903CECA29.h5
/home/ubuntu/MillionSongSubset/data/A/E/N/TRAENJZ12903CC901D.h5
/home/ubuntu/MillionSongSubset/data/A/E/N/TRAENRA128F934B998.h5
/home/ubuntu/MillionSongSubset/data/A/E/

/home/ubuntu/MillionSongSubset/data/A/E/Z/TRAEZAZ128F930608E.h5
/home/ubuntu/MillionSongSubset/data/A/E/Z/TRAEZMX128F425C3CE.h5
/home/ubuntu/MillionSongSubset/data/A/E/Z/TRAEZWE128F4244505.h5
/home/ubuntu/MillionSongSubset/data/A/E/Z/TRAEZKA128F4282B79.h5
/home/ubuntu/MillionSongSubset/data/A/E/Z/TRAEZLQ128F427F954.h5
/home/ubuntu/MillionSongSubset/data/A/E/Z/TRAEZFJ128F422A5C0.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHTU128F93353E5.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHYF128F148AE0C.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHBW12903CE1045.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHTT128F9303492.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHXW128F146F3C5.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHOI12903CD02C8.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHTK128F14A4706.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHLP128F933720D.h5
/home/ubuntu/MillionSongSubset/data/A/E/H/TRAEHMP128F92DCC42.h5
/home/ubuntu/MillionSongSubset/data/A/E/

/home/ubuntu/MillionSongSubset/data/A/N/J/TRANJMB128F42926E4.h5
/home/ubuntu/MillionSongSubset/data/A/N/J/TRANJSV128E0799689.h5
/home/ubuntu/MillionSongSubset/data/A/N/J/TRANJMO128F425C44D.h5
/home/ubuntu/MillionSongSubset/data/A/N/J/TRANJPA128F92CAE02.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANEMQ128F42A10A8.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANEQC12903D01CE6.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANEHV12903CD1E33.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANEQB128F4284978.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANELD128F4250FCF.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANELB128F92EF58A.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANELG128F424F264.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANEYV128F4269032.h5
/home/ubuntu/MillionSongSubset/data/A/N/E/TRANEMF128F42888C8.h5
/home/ubuntu/MillionSongSubset/data/A/N/N/TRANNJC128F429F7E3.h5
/home/ubuntu/MillionSongSubset/data/A/N/N/TRANNCH128F14B1A75.h5
/home/ubuntu/MillionSongSubset/data/A/N/

In [6]:
#msd_df = sqlContext.createDataFrame(zip(song_number, album_id, artist_latitude, artist_location, artist_longitude, artist_name, danceability, duration, energy, key_signature, key_signature_confidence, song_id, tempo, song_hotttnesss, time_signature, time_signature_confidence, title, year), schema=['song_number', 'album_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'danceability', 'duration', 'energy', 'key_signature', 'key_signature_confidence', 'song_id', 'tempo', 'song_hotttnesss', 'time_signature', 'time_signature_confidence', 'title', 'year'])

In [7]:
msd_df.show()

+-----------+--------+---------------+--------------------+----------------+--------------------+------------+---------+------+-------------+------------------------+--------------------+-------+-------------------+--------------+-------------------------+--------------------+----+
|song_number|album_id|artist_latitude|     artist_location|artist_longitude|         artist_name|danceability| duration|energy|key_signature|key_signature_confidence|             song_id|  tempo|    song_hotttnesss|time_signature|time_signature_confidence|               title|year|
+-----------+--------+---------------+--------------------+----------------+--------------------+------------+---------+------+-------------+------------------------+--------------------+-------+-------------------+--------------+-------------------------+--------------------+----+
|          1|  511272|               |                 b''|                |          b'Dickies'|         0.0|179.59138|   0.0|          2.0|          

In [8]:
msd_df.printSchema()

root
 |-- song_number: long (nullable = true)
 |-- album_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key_signature: double (nullable = true)
 |-- key_signature_confidence: double (nullable = true)
 |-- song_id: string (nullable = true)
 |-- tempo: double (nullable = true)
 |-- song_hotttnesss: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- time_signature_confidence: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)



In [9]:
start_time = time.time()

In [10]:
msd_df = msd_df.na.fill(0)
msd_df1 = msd_df.filter(msd_df['song_hotttnesss'] > 0).agg(avg(col("song_hotttnesss"))).collect()

In [11]:
average = rowUnpack(msd_df1)
average = average['avg(song_hotttnesss)']

msd_df_corrected = msd_df.withColumn("song_hotttnesss", \
              when(msd_df["song_hotttnesss"] == 0, average).otherwise(msd_df["song_hotttnesss"]))

In [12]:
msd_df_corrected.show()

+-----------+--------+---------------+--------------------+----------------+--------------------+------------+---------+------+-------------+------------------------+--------------------+-------+-------------------+--------------+-------------------------+--------------------+----+
|song_number|album_id|artist_latitude|     artist_location|artist_longitude|         artist_name|danceability| duration|energy|key_signature|key_signature_confidence|             song_id|  tempo|    song_hotttnesss|time_signature|time_signature_confidence|               title|year|
+-----------+--------+---------------+--------------------+----------------+--------------------+------------+---------+------+-------------+------------------------+--------------------+-------+-------------------+--------------+-------------------------+--------------------+----+
|          1|  511272|               |                 b''|                |          b'Dickies'|         0.0|179.59138|   0.0|          2.0|          

In [13]:
msd_df_sub = msd_df_corrected.select("duration","key_signature","tempo","time_signature","song_hotttnesss")
#msd_df_sub.show()

In [14]:
msd_sort = msd_df_sub.orderBy('duration', ascending=True)
#msd_sort.show()

In [15]:
(training_data, test_data) = msd_sort.randomSplit([0.7, 0.3])
training_data =training_data.rdd.map(lambda x: LabeledPoint(x[4], x[:4]))
test_data =test_data.rdd.map(lambda x: LabeledPoint(x[4], x[:4]))

#training_data.take(5)
#test_data.take(5)

In [16]:
model = DecisionTree.trainRegressor(training_data, categoricalFeaturesInfo={},
                                    impurity='variance', maxDepth=5, maxBins=32)

In [17]:
model_tree = model.toDebugString()

predictions = model.predict(test_data.map(lambda x: x.features))
labelsAndPredictions = test_data.map(lambda lp: lp.label).zip(predictions)

testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() / float(test_data.count())
end_time = time.time()
exec_time = end_time - start_time
print('Test Mean Squared Error = ' + str(testMSE))


In [18]:
print("--- %s seconds ---" % exec_time)


--- 29.352632999420166 seconds ---


In [19]:
input_cols = ["duration","key_signature","tempo","time_signature"]

for i, feat in enumerate(input_cols):
    model_tree = model_tree.replace('feature ' + str(i), feat)

print('Learned regression tree model: \n')
print(model_tree)

Learned regression tree model: 

DecisionTreeModel regressor of depth 5 with 53 nodes
  If (tempo <= 84.288)
   If (time_signature <= 0.5)
    If (key_signature <= 0.5)
     Predict: 0.45948195643562095
    Else (key_signature > 0.5)
     If (duration <= 233.65179)
      Predict: 0.6034409947366194
     Else (duration > 233.65179)
      Predict: 0.693947934490814
   Else (time_signature > 0.5)
    If (key_signature <= 0.5)
     If (time_signature <= 3.5)
      If (duration <= 275.526075)
       Predict: 0.40774666297059986
      Else (duration > 275.526075)
       Predict: 0.4720855202729366
     Else (time_signature > 3.5)
      If (duration <= 245.4722)
       Predict: 0.467170884496594
      Else (duration > 245.4722)
       Predict: 0.5124946748064468
    Else (key_signature > 0.5)
     If (tempo <= 66.869)
      If (duration <= 206.95465000000002)
       Predict: 0.4545419509003757
      Else (duration > 206.95465000000002)
       Predict: 0.43767098447536906
     Else (tempo > 66

In [20]:
spark_context.stop()

In [21]:
#model.save(spark_context, "/home/ubuntu/MillionSongLDSA")
#sameModel = DecisionTreeModel.load(sc, "target/tmp/myDecisionTreeRegressionModel")